In [1]:
import sys
import os
import numbers
import numpy as np
from IPython.display import Math, display
from typing import Union, List, Sequence, Optional

# Add the parent directory (project root) to sys.path
script_dir              = os.path.dirname(os.curdir)
parent_dir              = os.path.abspath(os.path.join(script_dir, '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import QES.general_python.common.binary as _bin_mod
from QES.general_python.lattices.honeycomb import HoneycombLattice
from QES.general_python.lattices.square import SquareLattice
from QES.general_python.lattices.lattice import LatticeBC, Lattice
from QES.general_python.common.timer import Timer
from QES.general_python.common.display import (
    display_state,
    display_operator_action,
    prepare_labels
)

from QES.Algebra.Operator.operator import OperatorTypeActing, Operator, JAX_AVAILABLE
#! Spin operators
import QES.Algebra.Operator.operators_spin as op_spin
#! Fermionic operators
import QES.Algebra.Operator.operators_spinless_fermions as op_sferm


#! Backends
if JAX_AVAILABLE:
    import jax
    import jax.numpy as jnp
else:
    jax = None
    jnp = None
    
backend = 'np'
lat     = SquareLattice(dim     = 1,
                        lx      = 5,
                        ly      = 1,
                        lz      = 1,
                        bc      = LatticeBC.PBC)

#! Create functions for testing

def test_operator_on_state(op           : Union[Operator, Sequence[Operator]],
                        lat             : Lattice,
                        state           : Union[int, np.ndarray, jnp.ndarray],
                        *,
                        ns              : Optional[int] = None,
                        op_acting       : "OperatorTypeActing" = OperatorTypeActing.Local,
                        op_label        = None,
                        to_bin          = None,
                        just_time       = False
                        ) -> None:
    r"""
    Pretty-print the action of *one or several* lattice operators
    on a basis state or wave-function.

    Parameters
    ----------
    op : Operator or sequence[Operator]
        The operator(s) Ô acting on 0, 1, or 2 sites.
    lat : Lattice
        Provides the number of sites ``lat.ns`` = :math:`N_s`.
    ns : int, optional
        Number of sites.  If *None*, uses ``lat.ns``.
    state : int | np.ndarray | jax.numpy.ndarray
        *Basis state* (integer encoding) or *wave-function* :math:`|\psi\rangle`.
    op_acting : OperatorTypeActing, default = ``Local``
        How Ô acts: Local (Ôᵢ), Correlation (Ôᵢⱼ), or Global (Ô).
    op_label : str | sequence[str], optional
        LaTeX label(s).  If *None*, uses ``op.name`` for every operator.
    to_bin : Callable[[int, int], str], optional
        Integer → binary-string formatter.  Defaults to
        ``lambda k,L: format(k, f'0{L}b')``.
    just_time : bool, default = False
        If True, only measure the time taken for the operation without
        displaying the results.  This is useful for benchmarking.

    Notes
    -----
    * For **integer states** we reproduce the coefficient table you had before.
    * For **array states** (NumPy / JAX) we show only the *first* non-zero
        coefficient returned by the operator.  Adjust if you need more detail.
    """
    # ------------------------------------------------------------------
    ops      = (op,) if not isinstance(op, Sequence) else tuple(op)
    is_int   = isinstance(state, (numbers.Integral, int, np.integer)) 
    ns       = lat.ns if ns is None else ns
    labels   = prepare_labels(ops, op_label) if not just_time else [''] * len(ops)
    def display_in(*args, **kwargs): 
        if not just_time:
            display(*args, **kwargs)
    
    # ------------------------------------------------------------------
    display_state(state,
                ns,
                label   = f"Initial integer state (Ns={ns})",
                to_bin  = to_bin,
                verbose = not just_time)

    # ------------------------------------------------------------------
    with Timer(verbose=True, precision='us', name="Operator action"):
        for cur_op, lab in zip(ops, labels):
            display_in(Math(fr"\text{{Operator: }} {lab}, \text{{typeacting}}: {op_acting}"))
            
            if op_acting == op_spin.OperatorTypeActing.Local or op_acting == op_sferm.OperatorTypeActing.Local:
                for i in range(ns):
                    display_in(Math(fr"\quad \text{{Site index: }} {i}"))
                    s, c = _dispatch(cur_op, state, lat, is_int, to_bin, lab, i=i, just_time=just_time)

            elif op_acting == OperatorTypeActing.Correlation or op_acting == op_sferm.OperatorTypeActing.Correlation:
                for i in range(ns):
                    for j in range(ns):
                        display_in(Math(fr"\text{{Site indices: }} {i}, {j}"))
                        s, c = _dispatch(cur_op, state, lat, is_int, to_bin, lab, i=i, j=j, just_time=just_time)

            elif op_acting == OperatorTypeActing.Global or op_acting == op_sferm.OperatorTypeActing.Global:
                s, c = _dispatch(cur_op, state, lat, is_int, to_bin, lab, just_time=just_time)
            else:
                raise ValueError(f"Operator acting type {op_acting!r} not supported.")
            
#------------------------------------------------------------------------
#! Helper functions
#------------------------------------------------------------------------

def _dispatch(op, state, lat, is_int, to_bin, lab, *, i=None, j=None, just_time=False):
    """
    Call *op* with the right signature and send its first output to
    `display_operator_action`.
    """
    
    state_act = state
    
    # call signature depends on OperatorTypeActing
    if i is None:                        # Global operator
        st_out, coeff = op(state_act)
    elif j is None:                      # Local operator
        if is_int:
            st_out, coeff = op(state_act, i)
        elif isinstance(state_act, (np.ndarray)):
            sites         = np.array([i])
            st_out, coeff = op(state_act, *sites)
        elif isinstance(state_act, (jnp.ndarray)):
            sites         = jnp.array([i])
            st_out, coeff = op(state_act, sites[0])
    else:                                # Correlation operator
        if is_int:
            st_out, coeff = op(state_act, i, j)
        elif isinstance(state_act, (np.ndarray)):
            sites         = np.array([i, j])
            st_out, coeff = op(state_act, *sites)
        elif isinstance(state_act, (jnp.ndarray)):
            sites         = jnp.array([i, j])
            st_out, coeff = op(state_act, sites[0], sites[1])
            
    #! choose what to show depending on state representation
    new_state = st_out
    new_coeff = coeff
    if not just_time:
        display_operator_action(f"\\quad \\quad {lab}",
                                i if j is None else (i, j),
                                state,
                                lat.ns,
                                new_state,
                                new_coeff,
                                to_bin = to_bin)
    return new_state, new_coeff

# ------------------------------------------------------------------

def initial_states(ns: int, display: bool = False) -> tuple:
    int_state = np.random.randint(0, 2**(ns%64), dtype=np.int32)
    if ns >= 64:
        np_state  = np.random.randint(0, 2, size=lat.ns).astype(np.float32)
        jnp_state = jnp.array(np_state)
    else:
        np_state  = _bin_mod.int2base_np(int_state, size = ns, value_true=1, value_false=0).astype(np.float32)
        jnp_state = jnp.array(np_state)
    
    if display:
        display_state(int_state, lat.ns,    label = "Integer state")
        display_state(np_state, lat.ns,     label = "NumPy state")
        display_state(jnp_state, lat.ns,    label = "JAX state")
    
    return int_state, np_state, jnp_state

# ------------------------------------------------------------------

09_05_2025_00-26_32 [INFO] Log file created: ./log/QES_09_05_2025_00-26_32.log
09_05_2025_00-26_32 [INFO] Log level set to: info
09_05_2025_00-26_32 [INFO] ############Global logger initialized.############
09_05_2025_00-26_32 [INFO] JAX backend available and successfully imported
09_05_2025_00-26_32 [INFO] 	JAX 64-bit precision enabled.
09_05_2025_00-26_32 [INFO] Setting JAX as the active backend.
09_05_2025_00-26_32 [INFO] **************************************************
09_05_2025_00-26_32 [INFO] Backend Configuration:
09_05_2025_00-26_32 [INFO] 		NumPy Version: 2.1.3
09_05_2025_00-26_32 [INFO] 		SciPy Version: 1.15.2
09_05_2025_00-26_32 [INFO] 		JAX Version: 0.5.3
09_05_2025_00-26_32 [INFO] 		Active Backend: jax
09_05_2025_00-26_32 [INFO] 			JAX Available: True
09_05_2025_00-26_32 [INFO] 			Default Seed: 42
09_05_2025_00-26_32 [INFO] 		JAX Backend Details:
09_05_2025_00-26_32 [INFO] 				Main Module: jax.numpy
09_05_2025_00-26_32 [INFO] 				Random Module: jax.random (+ PRNGKey)
09

## Spin operators $\sigma^x$, $\sigma^y$, $\sigma^z$

The Pauli matrices are defined as:
$$
\sigma^x = \begin{pmatrix}
0 & 1 \\
1 & 0
\end{pmatrix}, \quad
\sigma^y = \begin{pmatrix}
0 & -i \\
-i & 0
\end{pmatrix}, \quad
\sigma^z = \begin{pmatrix}
1 & 0 \\
0 & -1
\end{pmatrix}
$$

### a) Local operators

In [2]:
type_act = op_spin.OperatorTypeActing.Local

sig_x_l = op_spin.sig_x(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_y_l = op_spin.sig_y(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_z_l = op_spin.sig_z(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_p_l = op_spin.sig_p(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_m_l = op_spin.sig_m(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_pm_l = op_spin.sig_pm(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_mp_l = op_spin.sig_mp(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_x_sig_x_l = sig_x_l * sig_x_l

operators_l  = [sig_x_l, sig_y_l, sig_z_l, sig_p_l, sig_m_l, sig_pm_l, sig_mp_l, sig_x_sig_x_l]
labels_l     = ['\\sigma^x', '\\sigma^y', '\\sigma^z', 
                '\\sigma^+', '\\sigma^-', 
                '[\\sigma^+]\\sigma^{\\pm}', '[\\sigma^-]\\sigma^{\\mp}', '\\sigma^x\\sigma^x'][:len(operators_l)]
# create a set of states and test it
int_state, np_state, jnp_state = initial_states(lat.ns, display=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### i) Integer states

In [12]:
test_operator_on_state(
    op          = operators_l,
    lat         = lat,
    state       = int_state,
    op_acting   = type_act,
    op_label    = labels_l,
    to_bin      = None,
    just_time   = False
)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Operator action: 539822.292000 us


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function setitem>) found for signature:
 
 >>> setitem(array(int64, 1d, C), Literal[int](0), array(int64, 1d, C))
 
There are 16 candidate implementations:
[1m    - Of which 16 did not match due to:
    Overload of function 'setitem': File: <numerous>: Line N/A.
      With argument(s): '(array(int64, 1d, C), int64, array(int64, 1d, C))':[0m
[1m     No match.[0m
[0m
[0m[1mDuring: typing of staticsetitem at /Users/makskliczkowski/Codes/QuantumEigenSolver/Python/QES/Algebra/Operator/operators_spin.py (106)[0m
[1m
File "../QES/Algebra/Operator/operators_spin.py", line 106:[0m
[1mdef sigma_x_int_np(state, ns, sites, spin: bool = BACKEND_DEF_SPIN, spin_value=_SPIN):
    <source elided>
        coeff   *= spin_value
[1m    out_state[0]    = state
[0m    [1m^[0m[0m

[0m[1mDuring: Pass nopython_type_inference[0m

#### ii) Numpy states

In [5]:
test_operator_on_state(
    op          = operators_l,
    lat         = lat,
    state       = np_state,
    op_acting   = OperatorTypeActing.Local,
    op_label    = labels_l,
    to_bin      = None,
    just_time   = True
)


Operator action: 531.917000 us


#### iii) JAX states

In [4]:
test_operator_on_state(
    op          = operators_l,
    lat         = lat,
    state       = jnp_state,
    op_acting   = type_act,
    op_label    = labels_l,
    to_bin      = None,
    just_time   = False
)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Operator action: 194665.583000 us


### b) Correlation functions

In [13]:
type_act = op_spin.OperatorTypeActing.Correlation

sig_x_c = op_spin.sig_x(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_y_c = op_spin.sig_y(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_z_c = op_spin.sig_z(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_p_c = op_spin.sig_p(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_m_c = op_spin.sig_m(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_pm_c = op_spin.sig_pm(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
sig_mp_c = op_spin.sig_mp(
    lattice  = lat,
    type_act = type_act,
    spin     = False
)
operators_c  = [sig_x_c, sig_y_c, sig_z_c, sig_p_c, sig_m_c, sig_pm_c, sig_mp_c]
labels_c     = ['\\sigma^x', '\\sigma^y', '\\sigma^z', 
                '\\sigma^+', '\\sigma^-', 
                '[\\sigma^+]\\sigma^{\\pm}', '[\\sigma^-]\\sigma^{\\mp}'][:len(operators_c)]
# create a set of states and test it
int_state, np_state, jnp_state = initial_states(lat.ns, display=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### i) Integer states

In [14]:
test_operator_on_state(
    op          = operators_c,
    lat         = lat,
    state       = int_state,
    op_acting   = OperatorTypeActing.Correlation,
    op_label    = labels_c,
    to_bin      = None,
    just_time   = False
)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Operator action: 2147714.042000 us


## Creation and annihilation operators (spinless fermions)
The creation and annihilation operators for spinless fermions are defined as:
$$
c_i^\dagger = \begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}, \quad
c_i = \begin{pmatrix}
0 & 0 \\
0 & 1
\end{pmatrix}
$$
where $c_i^\dagger$ creates a fermion at site $i$ and $c_i$ annihilates a fermion at site $i$.

### a) Local operators

In [2]:
type_act = op_sferm.OperatorTypeActing.Local

c_l = op_sferm.c(
    lattice  = lat,
    ns       = lat.ns,
    type_act = type_act,
)
c_dag_l = op_sferm.cdag(
    lattice  = lat,
    ns       = lat.ns,
    type_act = type_act,
)
c_c_dag_l   = c_l * c_dag_l
n_l = op_sferm.n(
    lattice  = lat,
    ns       = lat.ns,
    type_act = type_act,
)
operators_l  = [c_l, c_dag_l, n_l, c_c_dag_l]
labels_l     = ['c', 'c^\\dag', 'n', '(c * c^\\dag)'][:len(operators_l)]

# create a set of states and test it
int_state, np_state, jnp_state = initial_states(lat.ns, display=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### i) Integer states

In [3]:
test_operator_on_state(
    op          = operators_l,
    lat         = lat,
    state       = int_state,
    op_acting   = op_sferm.OperatorTypeActing.Local,
    op_label    = labels_l,
    to_bin      = None,
    just_time   = False
)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Operator action: 889056.667000 us


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function setitem>) found for signature:
 
 >>> setitem(array(int64, 1d, C), Literal[int](0), array(int64, 1d, C))
 
There are 16 candidate implementations:
[1m  - Of which 16 did not match due to:
  Overload of function 'setitem': File: <numerous>: Line N/A.
    With argument(s): '(array(int64, 1d, C), int64, array(int64, 1d, C))':[0m
[1m   No match.[0m
[0m
[0m[1mDuring: typing of staticsetitem at /Users/makskliczkowski/Codes/QuantumEigenSolver/Python/QES/Algebra/Operator/operators_spinless_fermions.py (294)[0m
[1m
File "../QES/Algebra/Operator/operators_spinless_fermions.py", line 294:[0m
[1mdef c_int_np(state       : int,
    <source elided>
    out_coeff       = np.empty(1, dtype=_DEFAULT_FLOAT)
[1m    out_state[0]    = new_state
[0m    [1m^[0m[0m

[0m[1mDuring: Pass nopython_type_inference[0m

#### ii) Numpy states

In [4]:
test_operator_on_state(
    op          = operators_l,
    lat         = lat,
    state       = np_state,
    op_acting   = op_sferm.OperatorTypeActing.Local,
    op_label    = labels_l,
    to_bin      = None,
    just_time   = False
)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Operator action: 685834.625000 us


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mNo implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(list(array(float32, 1d, C))<iv=None>)
 
There are 2 candidate implementations:
[1m   - Of which 2 did not match due to:
   Overload in function 'impl_np_array': File: numba/np/arrayobj.py: Line 5546.
     With argument(s): '(list(array(float32, 1d, C))<iv=None>)':[0m
[1m    Rejected as the implementation raised a specific error:
      TypingError: Failed in nopython mode pipeline (step: nopython frontend)
    [1m[1m[1m[1mNo implementation of function Function(<intrinsic np_array>) found for signature:
     
     >>> np_array(list(array(float32, 1d, C))<iv=None>, none)
     
    There are 2 candidate implementations:
    [1m      - Of which 2 did not match due to:
          Intrinsic in function 'np_array': File: numba/np/arrayobj.py: Line 5520.
            With argument(s): '(list(array(float3

#### iii) JAX states

In [3]:
test_operator_on_state(
    op          = operators_l,
    lat         = lat,
    state       = jnp_state,
    op_acting   = op_sferm.OperatorTypeActing.Local,
    op_label    = labels_l,
    to_bin      = None,
    just_time   = False
)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Operator action: 504617.792000 us
